In [85]:
%matplotlib widget

from experiments import Experiment, _DefaultFloatLogSlider, _DefaultIntSlider

from translocation_models import TranslocationModel, \
    SC2R, SC2R2Loops, DefectiveSC2R, \
    DiscSpiral, DefectiveDiscSpiral

from ipywidgets import Widget, FloatLogSlider, FloatSlider, IntSlider, FloatRangeSlider, \
    HBox, VBox, HTML, Output, Layout
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.transforms import Affine2D
import pandas as pd
from pandas import DataFrame
import matplotlib as mpl
import networkx as nx
from networkx import DiGraph

import copy
from itertools import pairwise


In [100]:
# TODO do a super class with the common part of all NonIdeal models
class NonIdealSC2R(SC2R):
    """Non-ideal Sequential Clockwise/2-Residue Step translocation model.

    Non-ideal in the sense that at each state, there is a probability to
    leave the main loop, to a effective state 'out'.
    All k_in rates are similar, and remains only one free parameter k_out, 
    chosen to be out of TTT state.
    """

    def __init__(self) -> None:
        self.reference_state = 'TTT' # State from which the out rate is computed
        self.k_out = 1
        self.k_in = 1
        super().__init__()
    
    def _k_out(self, state: str) -> float:
        """Compute state out rate, constrained by the detailed balance.
        
        The constraint is computed based on the main loop.
        """
        rate = self.k_out
        try:
            path = nx.shortest_path(self._main_loop, state, self.reference_state)
        except nx.NetworkXNoPath:
            pass
        else:
            for u, v in pairwise(path):
                rate *= (self._main_loop.edges[u, v]['rate']()
                         / self._main_loop.edges[v, u]['rate']())
        return rate

    def _construct_kinetic_scheme(self, kinetic_scheme: DiGraph | None = None
                                  ) -> DiGraph:
        if not kinetic_scheme:
            kinetic_scheme = DiGraph()
        # Construct main loop
        kinetic_scheme = super()._construct_kinetic_scheme(kinetic_scheme)
        self._main_loop = copy.deepcopy(kinetic_scheme) # Used to compute _k_out
        # Add 'out' state
        kinetic_scheme.add_node(
            'out',
            probability=lambda: self._compute_probabilities()['out'])
        for node in kinetic_scheme.nodes:
            if node != 'out':
                kinetic_scheme.add_edges_from([
                    (node, 'out', {'rate': lambda node=node: self._k_out(node)}),
                    ('out', node, {'rate': lambda: self.k_in})])
        return kinetic_scheme
        

In [101]:
non_ideal_sc2r = NonIdealSC2R()
print([(u, v, data, data['rate']()) for u, v, data in non_ideal_sc2r.kinetic_scheme.edges(data=True)])

[('TTT', 'DTT', {'rate': <function SC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c2c00>, 'ATP': -1}, 1), ('TTT', 'TTD', {'rate': <function SC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c19e0>}, 0.1), ('TTT', 'out', {'rate': <function NonIdealSC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c07c0>}, 1), ('DTT', 'TTT', {'rate': <function SC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c0540>, 'ATP': 1}, 1), ('DTT', 'TTD', {'rate': <function SC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c2340>, 'position': 2}, 1), ('DTT', 'out', {'rate': <function NonIdealSC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c0680>}, 1.0), ('TTD', 'DTT', {'rate': <function SC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c1940>, 'position': -2}, 1.0), ('TTD', 'TTT', {'rate': <function SC2R._construct_kinetic_scheme.<locals>.<lambda> at 0x7f06178c1300>}, 1), ('TTD', 'out', {'rate': <function NonIdealSC2R._construct_kin